<a href="https://colab.research.google.com/github/Fagner608/MBA_apriori_with_R/blob/main/MBA_APRIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos trabalhar com um dataset de vendas on-line de alguns países da Europa. Ao final, teremos conhecimento das assossiações existentes entre os itens mais vendidos, o que poderá nos levar à um modelo de recomendação de compra.

## Sobre o conjunto de dados

Fonte: [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx)

Este dataset contém dados de varejo, e fornece dados de transações de um site com base no Reuni Unido.

## Motivação

Com base nas transações já registradas, queremos recomendar ao cliente compras que são bastante prováveis. Desta forma, estaremos aumentando o ticke-médio/cliente.

## Iniciando análise:

In [ ]:
# Instalando pacotes
install.packages('htmlwidgets')
install.packages('data.table')
install.packages('arules')
#install.packages('arulesViz')
install.packages('tidyr')
install.packages('reshape2')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘stringr’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘plyr’, ‘Rcpp’




In [ ]:
# Carregando pacotes

library(htmlwidgets)
library(data.table)
library(arules)
#library(arulesViz)
library(tidyr)
library(reshape2)

Loading required package: Matrix


Attaching package: ‘arules’


The following objects are masked from ‘package:base’:

    abbreviate, write



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


The following objects are masked from ‘package:data.table’:

    dcast, melt




Suprimindo warnings

In [ ]:
install.packages('ggplot2')
library(ggplot2)

install.packages('dplyr')
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:arules’:

    intersect, recode, setdiff, setequal, union


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
options(war = -1)

Extraindo dados

In [ ]:
download.file('https://raw.githubusercontent.com/Fagner608/MBA_apriori_with_R/main/retail.csv', 'dados.csv')

In [ ]:
#lendo dados

dados = read.csv('dados.csv')

dados

X,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<int>,<chr>
27,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583,France
28,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583,France
29,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583,France
30,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583,France
31,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583,France
32,536370,10002,INFLATABLE POLITICAL GLOBE,48,2010-12-01 08:45:00,0.85,12583,France
33,536370,21791,VINTAGE HEADS AND TAILS CARD GAME,24,2010-12-01 08:45:00,1.25,12583,France
34,536370,21035,SET/2 RED RETROSPOT TEA TOWELS,18,2010-12-01 08:45:00,2.95,12583,France
35,536370,22326,ROUND SNACK BOXES SET OF4 WOODLAND,24,2010-12-01 08:45:00,2.95,12583,France


In [ ]:
# Eliminando coluna que não serão usadas
dados = dados[, -c(1,3,5:9)]

In [ ]:
dados

InvoiceNo,Description
<chr>,<chr>
536370,ALARM CLOCK BAKELIKE PINK
536370,ALARM CLOCK BAKELIKE RED
536370,ALARM CLOCK BAKELIKE GREEN
536370,PANDA AND BUNNIES STICKER SHEET
536370,STARS GIFT TAPE
536370,INFLATABLE POLITICAL GLOBE
536370,VINTAGE HEADS AND TAILS CARD GAME
536370,SET/2 RED RETROSPOT TEA TOWELS
536370,ROUND SNACK BOXES SET OF4 WOODLAND


Vamos criar algumas funções, que receberão os atributos, e investigarão eventual padrão de dado que não paraça normal.

In [ ]:
# Função para investigar na's por colunas
na_cols = function(data.frame){

  na_s = colSums(is.na(data.frame))
  unique_v = apply(data.frame, 2, n_distinct)
  tipo_dados = apply(data.frame, 2, typeof)
  

  return(data.frame(na = na_s, valores_unicos = unique_v, tipo_dado = tipo_dados))
}

In [ ]:
na_cols(dados)

,na,valores_unicos,tipo_dado
,<dbl>,<int>,<chr>
InvoiceNo,0,461,character
Description,0,1565,character


Vamos verificar se existem caracteres alfabéticos no atributo invoiceNO

In [ ]:
indices = grep("[A-Za-z]", dados$InvoiceNo)

In [ ]:
length(indices)

[1] 149

In [ ]:
dados[indices, ]

,InvoiceNo,Description
,<chr>,<chr>
179,C537893,SILK PURSE BABUSHKA BLUE
180,C537893,CHILDS BREAKFAST SET SPACEBOY
181,C537893,DOLLY GIRL LUNCH BOX
355,C539104,LUNCH BAG DOLLY GIRL DESIGN
357,C539114,RECIPE BOX RETROSPOT
440,C540151,RED RETROSPOT CAKE STAND
441,C540156,RED RETROSPOT CAKE STAND
562,C540367,POSTAGE
696,C540853,CHILDS BREAKFAST SET SPACEBOY


São dados de compras canceladas. Vamos eliminar esses dados da análise

In [ ]:
dados_clean = dados[-indices, ]

In [ ]:
na_cols(dados_clean)

,na,valores_unicos,tipo_dado
,<dbl>,<int>,<chr>
InvoiceNo,0,392,character
Description,0,1564,character


Vamos atribuir o tipo correto aos dados

In [ ]:
dados_clean$InvoiceNo = as.integer(dados_clean$InvoiceNo)

In [ ]:
head(dados_clean)

,InvoiceNo,Description
,<int>,<chr>
1,536370,ALARM CLOCK BAKELIKE PINK
2,536370,ALARM CLOCK BAKELIKE RED
3,536370,ALARM CLOCK BAKELIKE GREEN
4,536370,PANDA AND BUNNIES STICKER SHEET
5,536370,STARS GIFT TAPE
6,536370,INFLATABLE POLITICAL GLOBE


Aplicando a função dcast() do pacote reshape2, vamos transformar os dados em uma matriz de presente, fazendo com que cada linha represente uma compra, e cada coluna represente um poduto, registando sua compra ou não.

In [ ]:
dados_transacoes = dcast(dados_clean, InvoiceNo ~ Description, fun.aggregate = length)

Using Description as value column: use value.var to override.



In [ ]:
head(dados_transacoes)

,InvoiceNo,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,⋯,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,536370,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,536852,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,536974,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,537065,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,537463,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,537468,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0


In [ ]:
dim(dados_transacoes)

[1]  392 1565

Elegendo os top-50 produtos vendidos

In [ ]:
top_50 = head(sort(colSums(dados_transacoes[, -1] == 0), decreasing = F), 50)

In [ ]:
dados_transacoes = dados_transacoes[, c('InvoiceNo', names(top_50))]

In [ ]:
dados_transacoes = dados_transacoes[, -2]

In [ ]:
head(dados_transacoes)

,InvoiceNo,RABBIT NIGHT LIGHT,RED TOADSTOOL LED NIGHT LIGHT,PLASTERS IN TIN WOODLAND ANIMALS,PLASTERS IN TIN CIRCUS PARADE,ROUND SNACK BOXES SET OF4 WOODLAND,LUNCH BAG RED RETROSPOT,LUNCH BOX WITH CUTLERY RETROSPOT,PLASTERS IN TIN SPACEBOY,RED RETROSPOT MINI CASES,⋯,SPACEBOY BIRTHDAY CARD,ASSORTED COLOUR MINI CASES,CHARLOTTE BAG APPLES DESIGN,CHILDRENS CUTLERY SPACEBOY,CIRCUS PARADE LUNCH BOX,COFFEE MUG APPLES DESIGN,LUNCH BOX I LOVE LONDON,RED HARMONICA IN BOX,RED RETROSPOT CHILDRENS UMBRELLA,SET OF 2 TEA TOWELS APPLE AND PEARS
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,536370,0,1,0,0,1,0,0,0,0,⋯,0,0,0,0,1,0,1,0,0,0
2,536852,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,536974,0,0,0,0,0,1,1,0,1,⋯,0,1,0,0,0,0,1,1,0,0
4,537065,0,0,1,0,1,1,1,1,1,⋯,0,1,0,0,0,0,0,0,0,0
5,537463,0,1,1,0,1,0,1,0,0,⋯,0,0,0,0,0,0,0,0,1,1
6,537468,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Trasnformando os dados em uma matriz

In [ ]:
dados_matrix = as.matrix(dados_transacoes[ , -1])

Casting para o tipo 'transaction'

In [ ]:
transacoes = as(dados_matrix, 'transactions')

Warning message in asMethod(object):
“matrix contains values other than 0 and 1! Setting all entries != 0 to 1.”


Aplicando apriori

In [ ]:
regras = apriori(transacoes,
                parameter = list(conf = 0.5, supp = 0.01, minlen = 3))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5    0.01      3
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 3 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[49 item(s), 392 transaction(s)] done [0.00s].
sorting and recoding items ... [49 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 4 5 6 7 8 9 10

Warning message in apriori(transacoes, parameter = list(conf = 0.5, supp = 0.01, :
“Mining stopped (maxlen reached). Only patterns up to a length of 10 returned!”


 done [0.01s].
writing ... [27563 rule(s)] done [0.01s].
creating S4 object  ... done [0.01s].


Filtrando regras redundantes

In [ ]:
regras_clean = regras[!is.redundant(regras)]

In [ ]:
length(regras_clean)

[1] 8805

Inspecionando as top_50 regras

In [ ]:
inspect(head(sort(regras_clean, by = 'support', decreasing = T),50))

É possível observar que a descrição dos produtos não traz apenas a sua definição (seu substantivo); os produtos vêm acompanhados de suas características, algumas delas irrelevantes, como cor, por exemplo.

Este resultado pode sim ter o seu valor, mas, o que queremos ver são as possíveis assossociações entre produtos.

Para isso, vamos retomar a análise do início, aplicando mais um tratamento, antes da transformação.

Vamos aproveitar os labels do top_50

In [ ]:
names(top_50)

[1] "POSTAGE"                             "RABBIT NIGHT LIGHT"                 
 [3] "RED TOADSTOOL LED NIGHT LIGHT"       "PLASTERS IN TIN WOODLAND ANIMALS"   
 [5] "PLASTERS IN TIN CIRCUS PARADE "      "ROUND SNACK BOXES SET OF4 WOODLAND "
 [7] "LUNCH BAG RED RETROSPOT"             "LUNCH BOX WITH CUTLERY RETROSPOT "  
 [9] "PLASTERS IN TIN SPACEBOY"            "RED RETROSPOT MINI CASES"           
[11] "SET/6 RED SPOTTY PAPER CUPS"         "SET/20 RED RETROSPOT PAPER NAPKINS "
[13] "SET/6 RED SPOTTY PAPER PLATES"       "LUNCH BAG APPLE DESIGN"             
[15] "REGENCY CAKESTAND 3 TIER"            "SPACEBOY LUNCH BOX "                
[17] "STRAWBERRY LUNCH BOX WITH CUTLERY"   "LUNCH BAG SPACEBOY DESIGN "         
[19] "LUNCH BAG WOODLAND"                  "ROUND SNACK BOXES SET OF 4 FRUITS " 
[21] "MINI PAINT SET VINTAGE "             "ALARM CLOCK BAKELIKE PINK"          
[23] "PACK OF 72 RETROSPOT CAKE CASES"     "DOLLY GIRL LUNCH BOX"               
[25] "ALARM CLOCK BAKELIKE GREEN"          "JUMBO BAG RED RETROSPOT"            
[27] "RED RETROSPOT CHARLOTTE BAG"         "ALARM CLOCK BAKELIKE RED "          
[29] "TEA PARTY BIRTHDAY CARD"             "RETROSPOT TEA SET CERAMIC 11 PC "   
[31] "LUNCH BAG DOLLY GIRL DESIGN"         "BAKING SET 9 PIECE RETROSPOT "      
[33] "PAPER BUNTING RETROSPOT"             "PLASTERS IN TIN STRONGMAN"          
[35] "JUMBO BAG WOODLAND ANIMALS"          "SET/10 RED POLKADOT PARTY CANDLES"  
[37] "WOODLAND CHARLOTTE BAG"              "4 TRADITIONAL SPINNING TOPS"        
[39] "CHILDRENS CUTLERY DOLLY GIRL "       "RED RETROSPOT PICNIC BAG"           
[41] "SPACEBOY BIRTHDAY CARD"              "ASSORTED COLOUR MINI CASES"         
[43] "CHARLOTTE BAG APPLES DESIGN"         "CHILDRENS CUTLERY SPACEBOY "        
[45] "CIRCUS PARADE LUNCH BOX "            "COFFEE MUG APPLES DESIGN"           
[47] "LUNCH BOX I LOVE LONDON"             "RED  HARMONICA IN BOX "             
[49] "RED RETROSPOT CHILDRENS UMBRELLA"    "SET OF 2 TEA TOWELS APPLE AND PEARS"

Aplicando filtro aos dados


In [ ]:
dados_clean_2 = dados_clean

Agora, vamos aplicar à  descrição dos produtos uma função, visando padronizar sua definição, descartando caracteristicas que não nos interessam no momentos

In [ ]:
lista_produtos = c('NIGHT LIGHT', 'PLASTERS', 'SNACK BOXES',
                  'LUNCH BAG', 'LUNCH BOX', 'MINI CASES', 'PAPER CUPS',
                  'PAPER NAPKINS', 'PAPER PLATES', 'REGENCY CAKESTAND', 'ALARM CLOCK',
                  'CAKE CASES', 'JUMBO BAG', 'BIRTHDAY CARD', 'TEA SET', 'BAKING SET', 'PAPER BUNTING',
                  'RED POLKADOT PARTY CANDLES', 'SPINNING TOPS', 'CHILDRENS CUTLERY', 'PICNIC BAG', 'CHARLOTTE BAG',
                  'COFFEE MUG', 'TEA TOWELS', 'CHILDRENS UMBRELLA')

In [ ]:
for(i in lista_produtos){

  indices = grep(i, dados_clean_2$Description)
  dados_clean_2$Description[indices] = i

}

In [ ]:
head(dados_clean_2)

,InvoiceNo,Description
,<int>,<chr>
1,536370,ALARM CLOCK
2,536370,ALARM CLOCK
3,536370,ALARM CLOCK
4,536370,PANDA AND BUNNIES STICKER SHEET
5,536370,STARS GIFT TAPE
6,536370,INFLATABLE POLITICAL GLOBE


Filtrando produtos, para conter somente o da nossa lista

In [ ]:
dados_clean_2 = dados_clean_2[dados_clean_2$Description == lista_produtos, ]

Warning message in dados_clean_2$Description == lista_produtos:
“longer object length is not a multiple of shorter object length”


In [ ]:
dim(dados_clean_2)

[1] 108   2

In [ ]:
dados_transacoes = dcast(dados_clean_2, InvoiceNo ~ Description, fun.aggregate = length)

Using Description as value column: use value.var to override.



Reorganizando colunas

In [ ]:
reordered = sort(colSums(dados_transacoes[, -1] == 0), decreasing = F)

dados_transacoes = dados_transacoes[, c('InvoiceNo', names(reordered))]

In [ ]:
head(dados_transacoes)

,InvoiceNo,LUNCH BAG,ALARM CLOCK,JUMBO BAG,BIRTHDAY CARD,CHILDRENS CUTLERY,COFFEE MUG,LUNCH BOX,CHARLOTTE BAG,NIGHT LIGHT,⋯,BAKING SET,CHILDRENS UMBRELLA,PAPER CUPS,PICNIC BAG,TEA SET,TEA TOWELS,PAPER BUNTING,RED POLKADOT PARTY CANDLES,REGENCY CAKESTAND,SPINNING TOPS
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,537065,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,537463,0,0,0,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,539607,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,540178,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
5,540521,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
6,540835,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0


In [ ]:
dados_matrix = as.matrix(dados_transacoes[ , -1])

In [ ]:
transacoes = as(dados_matrix, 'transactions')

In [ ]:
regras = apriori(transacoes,
                parameter = list(conf = 0.5, supp = 0.01, minlen = 3))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5    0.01      3
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[25 item(s), 89 transaction(s)] done [0.00s].
sorting and recoding items ... [25 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 4 5 done [0.00s].
writing ... [63 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [ ]:
regras_clean = regras[!is.redundant(regras)]

In [ ]:
length(regras_clean)

[1] 38

In [ ]:
inspect(sort(regras_clean, by = 'support', decreasing = T))

     lhs                                 rhs                 support   
[1]  {COFFEE MUG, PAPER BUNTING}      => {LUNCH BOX}         0.01123596
[2]  {LUNCH BOX, PAPER BUNTING}       => {COFFEE MUG}        0.01123596
[3]  {COFFEE MUG, LUNCH BOX}          => {PAPER BUNTING}     0.01123596
[4]  {PAPER NAPKINS, PAPER CUPS}      => {PAPER PLATES}      0.01123596
[5]  {PAPER PLATES, PAPER CUPS}       => {PAPER NAPKINS}     0.01123596
[6]  {PAPER PLATES, PAPER NAPKINS}    => {PAPER CUPS}        0.01123596
[7]  {JUMBO BAG, CAKE CASES}          => {CHILDRENS CUTLERY} 0.01123596
[8]  {CHILDRENS CUTLERY, CAKE CASES}  => {JUMBO BAG}         0.01123596
[9]  {JUMBO BAG, CHILDRENS CUTLERY}   => {CAKE CASES}        0.01123596
[10] {CHILDRENS CUTLERY, NIGHT LIGHT} => {ALARM CLOCK}       0.01123596
[11] {ALARM CLOCK, NIGHT LIGHT}       => {CHILDRENS CUTLERY} 0.01123596
[12] {JUMBO BAG, SNACK BOXES}         => {CHILDRENS CUTLERY} 0.01123596
[13] {CHILDRENS CUTLERY, SNACK BOXES} => {JUMBO BAG}         0.0

análise em andamento